In [ ]:
%load_ext autoreload
%env TFDS_DATA_DIR=/Users/paul/scratch/tensorflow_datasets

import sys
sys.path.append("../")

import numpy as np
from tensorflow import keras
from oodeel.methods import MLS
from oodeel.methods import DKNN
import tensorflow as tf
import matplotlib.pyplot as plt

from oodeel.eval.metrics import bench_metrics, get_curve
from oodeel.datasets import DataHandler

def normalize(x):
    return x/255


## Data and model loading
ID data: MNIST, OOD data: Fashion MNIST

In [ ]:

model = tf.keras.models.load_model("../saved_models/mnist_model")

In [ ]:


data_handler = DataHandler()
ds1 = data_handler.load_tfds('mnist', preprocess=True, preprocessing_fun=normalize)
ds2 = data_handler.load_tfds('fashion_mnist', preprocess=True, preprocessing_fun=normalize)
x_id = ds1["test"]
x_ood = ds2["test"]

## Maximum Logit Score
(A good classifier is all you need ?)

Example of method that does not need ID data to compute statistics to compare against

In [ ]:
%autoreload 2

data_handler = DataHandler()
ds1 = data_handler.load_tfds('mnist', preprocess=True, preprocessing_fun=normalize)
ds2 = data_handler.load_tfds('fashion_mnist', preprocess=True, preprocessing_fun=normalize)
x_id = ds1["test"]
x_ood = ds2["test"]


x_test = data_handler.merge_tfds(x_id, x_ood, shuffle=True).batch(100)

In [ ]:

%autoreload 2

oodmodel = MLS()
oodmodel.fit(model)
scores = oodmodel.score(x_test)

Score visualization

In [ ]:
%autoreload 2

plt.figure(figsize=(7,5))

scores_id = oodmodel.score(x_id.batch(100))
scores_ood = oodmodel.score(x_ood.batch(100))
plt.hist((scores_ood, scores_id), bins=10, color=("blue", "orange"), label=("ood", "id"), density=True)
plt.legend()
plt.show()

## Deep KNN
Example of method that needs ID data to compute statistics to compare against

In [ ]:
data_handler = DataHandler()
ds1 = data_handler.load_tfds('mnist', preprocess=True, preprocessing_fun=normalize)
ds2 = data_handler.load_tfds('fashion_mnist', preprocess=True, preprocessing_fun=normalize)
x_id = ds1["test"]
x_ood = ds2["test"]
x_train = ds1["train"]


x_test = data_handler.merge_tfds(x_id.take(1000), x_ood.take(1000), shuffle='True').batch(100)


In [ ]:
%autoreload 2
oodmodel = DKNN()
oodmodel.fit(model, x_train.take(10000).batch(100))
scores = oodmodel.score(x_test)

Score visualization

In [ ]:
%autoreload 2
plt.figure(figsize=(8,5))


#x_id, y_id = data_handler.convert_to_numpy(x_id)
#x_ood, y_ood = data_handler.convert_to_numpy(x_ood)
scores_id = oodmodel.score(x_id.take(1000).batch(100))
scores_ood = oodmodel.score(x_ood.take(1000).batch(100))
plt.hist((scores_ood, scores_id), bins=30, color=("blue", "orange"), label=("ood", "id"), density=True)
plt.legend()
plt.show()

In [ ]:
%autoreload 2
from oodeel.methods import ODIN

data_handler = DataHandler()
ds1 = data_handler.load_tfds('mnist', preprocess=True, preprocessing_fun=normalize)
ds2 = data_handler.load_tfds('fashion_mnist', preprocess=True, preprocessing_fun=normalize)
x_id = ds1["test"]
x_ood = ds2["test"]
x_train = ds1["train"]


oodmodel = ODIN()
oodmodel.fit(model)



plt.figure(figsize=(8,5))
#x_idn, y_id = data_handler.convert_to_numpy(x_id)
#x_oodn, y_ood = data_handler.convert_to_numpy(x_ood)
scores_id = oodmodel.score(x_id.batch(100))
scores_ood = oodmodel.score(x_ood.batch(100))
plt.hist((scores_ood, scores_id), bins=30, color=("blue", "orange"), label=("ood", "id"), density=True)
plt.legend()
plt.show()